<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/HF_gpt_j_6b_Moderation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">LLM Hands On Course</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Create a Moderation system with LangChain and Hugging Face.</h2>
    <p>by <b>Pere Martra</b></p>
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

# How To Create a Moderation System Using LangChain & Hugging Face.

This notebook continues the One created with models from OpenAI:
https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/LangChain_OpenAI_Moderation_System.ipynb


We are going to create a Moderation System based in a Model from Hugging Face.

First the Model reads the User comments and answer them.

Then the same Model receives the answer and identify any kind of negativity modifiyng if necessary the comment.

The intention is to  prevent a text entry by the user from influencing a negative or out-of-tone response from the comment system.


Of course the notebook using the OpenAI Models has performed much better than this one.

With the OpenAI API you can use much more powerful models than what we can load on our local machine or in Google Colab.

The Notebook has been tested with many models, I have finally left two. One of them can be run in the free version of Colab, while the larger one should be run using the PRO version of Colab.
You can download the notebook and run it on your machine, you will need a GPU with 16 MB of memory.

In [ ]:
#Install de LangChain and openai libraries.
%pip install langchain
%pip install openai
%pip install transformers

## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.
* LLMChain: To create chains, where the prompts or the results can pass from one step to another inside the chain.

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [ ]:
import torch
import os
import numpy as np

In [ ]:
#model_id = "EleutherAI/gpt-neo-2.7B" #free version
model_id = "EleutherAI/gpt-j-6b" #Colab PRO

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    eos_token_id = int(tokenizer.convert_tokens_to_ids('.')),
    device_map='auto'
)
assistant_llm = HuggingFacePipeline(pipeline=pipe)

Create the template for the first model called assistant.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment. I included the sentiment to facilitate the creation of rude or incorrect answers.

Note the **eos_token_id**, I incorporated it to help the Model stop generating text. It has a tendency to put much more text than necessary.

In [ ]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} social media post commenter, you will respond to the following post
Post: "{customer_request}"
Comment:
"""

In [ ]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the assistant_prompt_template and the model. The model will receive the prompt generated with the prompt_template.

In [ ]:
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="assistant_response",
    verbose=False
)
#the output of the formatted prompt will pass directly to the LLM.

To execute the chain created it's necessary to call the .run method of the chain, and pass the variables necessaries.

In our case: customer_request and sentiment.

In [ ]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #callint the .run method from the chain created Above.
    assistant_response = assistant_chain.run(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [ ]:
# This the customer request, or customere comment in the forum moderated by the agent.
# feel free to update it.
customer_request = """Your product is a piece of shit. I want my money back!"""

In [ ]:
# Our assistatnt working in 'nice' mode.
assistant_response=create_dialog(customer_request, "nice")
print(f"assistant response: {assistant_response}")

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.


assistant response: 
"Thank you for the information.


In [ ]:
#Our assistant running in rude mode.
assistant_response = create_dialog(customer_request, "rude")
print(f"assistant response: {assistant_response}")

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.


assistant response: 
"Dude just like my old t shirt you still have an open anus."

"No, I don't take all my money back.


As you can see the answers we obtain is really far from being polite and we can't publish this messages to the forum, especially if they come from our company's AI assistant.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.

In [ ]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform it to positive. Avoid any negative words.
If it is nice, you will let it remain as is and repeat it word for word.
###
Original comment: {comment_to_moderate}
###
Edited comment:"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template
)

In [ ]:
moderator_llm = assistant_llm

In [ ]:
#We build the chain for the moderator.
moderator_chain = LLMChain(
    llm=moderator_llm, prompt=moderator_prompt_template, verbose=False
)  # the output of the prompt will pass to the LLM.

In [ ]:
# To run our chain we use the .run() command
moderator_says = moderator_chain.run({"comment_to_moderate": assistant_response})

print(f"moderator_says: {moderator_says}")

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.


moderator_says:  
"Dude, just like the old t-shirt you still have an open anus."

"No, I don't take all my money back.


Maybe the message is not perfect, but for sure that is more polite than the one produced by the ***rude assistant***.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.

First we create two chain, one for each pair of prompt and model.

In [ ]:
#The optput of the first chain must coincide with one of the parameters of the second chain.
#The parameter is defined in the prompt_template.
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="comment_to_moderate",
    verbose=False,
)

#verbose True because we want to see the intermediate messages.
moderator_chain = LLMChain(
    llm=moderator_llm,
    prompt=moderator_prompt_template,
    verbose=True
)

**SequentialChain** is used to link different chains and parameters.

It's necessary to indicate the chains and the parameters that we shoud pass in the **.run** method.

In [ ]:
from langchain.chains import SequentialChain

# Creating the SequentialChain class indicating chains and parameters.
assistant_moderated_chain = SequentialChain(
    chains=[assistant_chain, moderator_chain],
    input_variables=["sentiment", "customer_request"],
    verbose=True,
)

Lets use our Moderating System!

In [ ]:
# We can now run the chain.
assistant_moderated_chain.run({"sentiment": "rude", "customer_request": customer_request})

Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.




> Entering new SequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:13 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform it to positive. Avoid any negative words.
If it is nice, you will let it remain as is and repeat it word for word.
###
Original comment: I have to say...if your friend paid for this product to YOU...then you need to spend the money on your own problems/solution in the first place, before you go around telling everyone about how shitty this product is.
###
Edited comment:

> Finished chain.

> Finished chain.


' I have to say.... If your friend paid for this product to YOU...then _you_ need to spend the money on your own personal problems/solution in the first place before you go around telling everyone on here about how shitty this product is.'

Sometimes the Moderator works fine, sometimes not. The Model is not able to detect correctly the sentiment, and if it detects the sentimet the comment modified can be worse than the origina or have no sense at all.

The model used is far to be a state of art Model, is really far from what we can obtain with model offered by OpenAI, or OpenSource models like LLAMA.

If you want to check how this same solutions works with OpenAI API check this notebook: https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/LangChain_OpenAI_Moderation_System.ipynb